####  Холодные звонки

Классификатор только на основе телефонного номера. Без интернет логов.

In [6]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
import datetime
sc.stop()
conf = SparkConf().set("spark.executor.instances", 8).set("spark.driver.maxResultSize", "4g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)



In [2]:
create_table_query = '''
create table user_kposminin.ccalls_sig_2 as
select 
  c.ymd,
  c.phone_num,
  c.approve, 
  c.full_app, 
  c.stop_cond, 
  if(c.status in ('Успешно','Уже оставил заявку','Действующий клиент'),1,0) as target,
  if(c.status in ('Нет потребности в кредите','Не требуется сейчас','Попросил не звонить/Претензионный'),1,0) as no_need,
  if(c.status in ('Недозвон','Не отвечает','Левый контакт','Занято','Автоответчик','Недоступен','Ошибка телефонии',
                'Сбрасывает','Клиент умер','Звонок сорвался',''),1,0) as not_started,
  c.util_30,           
  c.status, 
  from_unixtime(unix_timestamp(dt), 'u') as weekday,
  hour(dt) as raw_hour,
  trim(pc.provider) as mob_provider,
  r.ind,
  r.pop_country_share, 
  r.pop_city_share, 
  r.population / r.area_sq_km as density,
  r.area_sq_km,
  trim(r.federal_district) as federal_district, 
  r.avg_salary_2015_rub, 
  r.utc_time_zone_val
from
  user_kposminin.cold_calls_4 c
  left join hermes.phone_codes pc on trim(pc.phone_code) = substr(c.phone_num,2,9)
  left join (select trim(pc_name) as pc_name, max(reg_stat_ind) as reg_stat_ind from user_kposminin.tmp_pc_2 group by  trim(pc_name)) tmp on tmp.pc_name = trim(pc.region)
  left join hermes.region_stat r on r.ind = tmp.reg_stat_ind
;
'''

In [4]:
read_query = '''
select * from user_kposminin.ccalls_sig_2
where not ind is Null
'''


data = hc.sql(read_query) 

#train_data = data \
#    .filter("ymd < '2016-05-15'") \
#    .toPandas()

test_data = data \
    .filter("ymd >= '2016-06-30'") \
    .toPandas()

In [16]:
import datetime
#train_data[train_data['ymd'] > datetime.datetime.strptime('2016-05-15','%Y-%m-%d').date()].shape
train_data = pd.concat([
        train_data,
        test_data[test_data['ymd'] <= datetime.datetime.strptime('2016-06-30','%Y-%m-%d').date()]
    ]).reset_index(drop = True)
test_data =  test_data[test_data['ymd'] > datetime.datetime.strptime('2016-06-30','%Y-%m-%d').date()]

In [6]:
for col in ['mob_provider','federal_district']:
    for c in pd.concat([train_data,test_data])[col].dropna().unique():
        train_data[col + '_' + c] = train_data[col].map(lambda v: 1 * (c == v))
        test_data[col + '_' + c]  =  test_data[col].map(lambda v: 1 * (c == v))

##for col in ['ind']:
#    for c in pd.concat([train_data,test_data])[col].dropna().unique():
#        train_data[col + '_' + str(c)] = train_data[col].map(lambda v: 1 * (c == v))
#        test_data[col + '_' + str(c)]  =  test_data[col].map(lambda v: 1 * (c == v))


In [27]:
for col in ['ind']:
    for c in pd.concat([train_data,test_data])[col].dropna().unique():
        train_data[col + '_' + str(c)] = train_data[col].map(lambda v:  (c == v))
        test_data[col + '_' + str(c)]  =  test_data[col].map(lambda v:  (c == v))


KeyboardInterrupt: 

In [7]:
#feat_cols = [c for c in train_data.columns if not c in [u'ymd',u'phone_num',u'approve',u'full_app',u'stop_cond',
#                u'target',u'no_need', u'not_started',u'util_30',u'status',u'mob_provider','federal_district','ind']
#             ]
feat_cols = [c for c in train_data.columns if not c in [u'ymd',u'phone_num',u'approve',u'full_app',u'stop_cond',
                u'target',u'no_need', u'not_started',u'util_30',u'status',u'mob_provider','federal_district','ind']
             and  not 'ind' in c]
#feat_cols = [u'weekday',u'raw_hour', u'pop_country_share',u'pop_city_share', u'density', u'area_sq_km',
#       u'avg_salary_2015_rub', u'utc_time_zone_val']
label_col = 'no_need'

(7672070, 120)

In [34]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean')
X, y  = imp.fit_transform(train_data[feat_cols]), train_data[label_col]
Xt,yt = imp.transform(test_data[feat_cols])     , test_data[label_col]


In [35]:
import sklearn
import xgboost as xgb

clfXGB1 = xgb.XGBClassifier()
clfXGB1.fit(X,y)
print('{0} AUCROC test: {1}. train {2}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = yt, 
                  y_score = [e[1] for e in clfXGB1.predict_proba(Xt)]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = y, 
                  y_score = [e[1] for e in clfXGB1.predict_proba(X)]
                )                                              
))

XGBoost AUCROC test: 0.541284669833. train 0.544120042319


In [36]:
import sklearn.linear_model
mLR = sklearn.linear_model.LogisticRegression(penalty = 'l1',C = 1, class_weight = 'auto')
mLR.fit(X,y)
print('{0} AUCROC: {1}'.format('LogisticRegression',sklearn.metrics.roc_auc_score(
                y_true = yt, 
                y_score = [e[1] for e in mLR.predict_proba(Xt)]
    )))

LogisticRegression AUCROC: 0.528063791206


/opt/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


In [38]:
test_data[label_col + '_XGBoost']= [e[1] for e in clfXGB1.predict_proba(Xt)]
train_data[label_col + '_XGBoost']= [e[1] for e in clfXGB1.predict_proba(X)]

In [39]:
label_col = 'not_started'
y  = train_data[label_col]
yt = test_data[label_col]

In [40]:
clfXGB2 = xgb.XGBClassifier()
clfXGB2.fit(X,y)
print('{0} AUCROC test: {1}. train {2}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = yt, 
                  y_score = [e[1] for e in clfXGB2.predict_proba(Xt)]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = y, 
                  y_score = [e[1] for e in clfXGB2.predict_proba(X)]
                )                                              
))

XGBoost AUCROC test: 0.550929662643. train 0.551733018347


In [41]:
test_data[label_col + '_XGBoost']= [e[1] for e in clfXGB2.predict_proba(Xt)]
train_data[label_col + '_XGBoost']= [e[1] for e in clfXGB2.predict_proba(X)]

In [42]:
label_col = 'target'
y  = train_data[label_col]
yt = test_data[label_col]

In [43]:
clfXGB3 = xgb.XGBClassifier()
clfXGB3.fit(X,y)
print('{0} AUCROC test: {1}. train {2}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = yt, 
                  y_score = [e[1] for e in clfXGB3.predict_proba(Xt)]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = y, 
                  y_score = [e[1] for e in clfXGB3.predict_proba(X)]
                )                                              
))

XGBoost AUCROC test: 0.548025901316. train 0.545155795906


In [44]:
test_data[label_col + '_XGBoost']= [e[1] for e in clfXGB3.predict_proba(Xt)]
train_data[label_col + '_XGBoost']= [e[1] for e in clfXGB3.predict_proba(X)]

In [45]:
feat_cols = [c for c in train_data.columns if not c in [u'ymd',u'phone_num',u'approve',u'full_app',u'stop_cond',
                u'target',u'no_need', u'not_started',u'util_30',u'status',u'mob_provider','federal_district','ind']
             and  not 'ind' in c]
label_col = 'full_app'
imp = Imputer(missing_values='NaN', strategy='mean')
X, y  = imp.fit_transform(train_data[feat_cols]), train_data[label_col]
Xt,yt = imp.transform(test_data[feat_cols])     , test_data[label_col]


In [46]:
clfXGB4 = xgb.XGBClassifier()
clfXGB4.fit(X,y)
print('{0} AUCROC test: {1}. train {2}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = yt, 
                  y_score = [e[1] for e in clfXGB4.predict_proba(Xt)]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = y, 
                  y_score = [e[1] for e in clfXGB4.predict_proba(X)]
                )                                              
))

XGBoost AUCROC test: 0.555699269119. train 0.553774011266


In [47]:
print('{0} AUCROC test: {1}. train {2}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = yt, 
                  y_score = [e[1] for e in clfXGB4.predict_proba(Xt)]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = y, 
                  y_score = [e[1] for e in clfXGB4.predict_proba(X)]
                )                                              
))

XGBoost AUCROC test: 0.555699269119. train 0.553774011266


In [48]:
test_data[label_col + '_XGBoost']= [e[1] for e in clfXGB4.predict_proba(Xt)]
train_data[label_col + '_XGBoost']= [e[1] for e in clfXGB4.predict_proba(X)]

In [49]:
feat_cols = [c for c in train_data.columns if not c in [u'ymd',u'phone_num',u'approve',u'full_app',u'stop_cond',
                u'target',u'no_need', u'not_started',u'util_30',u'status',u'mob_provider','federal_district','ind']
             and  not 'ind' in c]
label_col = 'approve'
imp = Imputer(missing_values='NaN', strategy='mean')
X, y  = imp.fit_transform(train_data[feat_cols]), train_data[label_col]
Xt,yt = imp.transform(test_data[feat_cols])     , test_data[label_col]


In [50]:
clfXGB5 = xgb.XGBClassifier()
clfXGB5.fit(X,y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [51]:
print('{0} AUCROC test: {1}. train {2}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = yt, 
                  y_score = [e[1] for e in clfXGB5.predict_proba(Xt)]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = y, 
                  y_score = [e[1] for e in clfXGB5.predict_proba(X)]
                )                                              
))

XGBoost AUCROC test: 0.535909990846. train 0.53642460477


In [52]:
from sklearn.ensemble import AdaBoostClassifier

clfAB = AdaBoostClassifier(learning_rate = 1, n_estimators = 100)
clfAB.fit(X,y)
print('{0} AUCROC: {1}'.format('AdaBoostClassifier',sklearn.metrics.roc_auc_score(
                y_true = yt, 
                y_score = [e[1] for e in clfAB.predict_proba(Xt)]
    )))

AdaBoostClassifier AUCROC: 0.533817790316


In [53]:
test_data[label_col + '_XGBoost']= [e[1] for e in clfXGB5.predict_proba(Xt)]
train_data[label_col + '_XGBoost']= [e[1] for e in clfXGB5.predict_proba(X)]

In [54]:
train_data.to_csv('./data/train_ccalls1.csv',encoding = 'utf-8')
test_data.to_csv('./data/test_ccalls1.csv',encoding = 'utf-8')

KeyboardInterrupt: 

In [3]:
train_data = pd.read_csv('./data/train_ccalls1.csv',encoding = 'utf-8')
test_data = pd.read_csv('./data/test_ccalls1.csv',encoding = 'utf-8')

In [8]:
train_data['train_sample'] = 1
test_data['train_sample'] = 0
cols_to_export = [c for c in train_data.columns if not 'ind_' in c and not 'mob_provider_' in c and not 'federal_district_' in c]
#cols_to_export

['Unnamed: 0',
 u'ymd',
 u'phone_num',
 u'approve',
 u'full_app',
 u'stop_cond',
 u'target',
 u'no_need',
 u'not_started',
 u'util_30',
 u'status',
 u'weekday',
 u'raw_hour',
 u'mob_provider',
 u'ind',
 u'pop_country_share',
 u'pop_city_share',
 u'density',
 u'area_sq_km',
 u'federal_district',
 u'avg_salary_2015_rub',
 u'utc_time_zone_val',
 u'no_need_XGBoost',
 u'not_started_XGBoost',
 u'target_XGBoost',
 u'full_app_XGBoost',
 u'approve_XGBoost',
 'train_sample']

In [12]:
hc.sql('drop table if exists train_data1')
hc.registerDataFrameAsTable(hc.createDataFrame(train_data[cols_to_export]), 'train_data1')
hc.sql('drop table if exists user_kposminin.ccalls_phone_data_only')
hc.sql('create table user_kposminin.ccalls_phone_data_only as select * from train_data1')

DataFrame[]

In [13]:
train_data[cols_to_export].to_csv('./data/train_ccalls1.csv',encoding = 'utf-8')
test_data[cols_to_export].to_csv('./data/test_ccalls1.csv',encoding = 'utf-8')

In [57]:
import pickle
XGB = {
    'XGBoost_no_need': clfXGB1,
    'XGBoost_not_started': clfXGB2,
    'XGBoost_target': clfXGB3,
    'XGBoost_full_app': clfXGB4,
    'XGBoost_approve': clfXGB5    
}
pickle.dump(XGB,open('xgboosters.pck','w'))

In [1]:
import pickle
XGB = pickle.load(open('xgboosters.pck','r'))

In [2]:
new_data = hc.sql('select * from user_kposminin.ccalls_sig_test_1').toPandas()

In [3]:
new_data.head()

,ymd,phone_num,weekday,raw_hour,mob_provider,ind,pop_country_share,pop_city_share,density,area_sq_km,federal_district,avg_salary_2015_rub,utc_time_zone_val
0,2016-08-05,83519024339,5,6,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,2016-08-19,85713599098,5,12,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,2016-08-17,89000000015,3,16,Tele2,2,0.0376,0.5432,73.045029,75485,ЮФО,31376,3
3,2016-08-21,89000000101,7,16,Tele2,2,0.0376,0.5432,73.045029,75485,ЮФО,31376,3
4,2016-08-14,89000003555,7,8,Tele2,2,0.0376,0.5432,73.045029,75485,ЮФО,31376,3


In [4]:
for col in ['mob_provider','federal_district']:
    for c in new_data[col].dropna().unique():
        new_data[col + '_' + c] = new_data[col].map(lambda v: 1 * (c == v))


In [10]:
for col in ['ind']:
    for c in new_data[col].dropna().unique():
        new_data[col + '_' + str(c)] = new_data[col].map(lambda v:  (c == v))


In [16]:
new_data

,ymd,phone_num,weekday,raw_hour,mob_provider,ind,pop_country_share,pop_city_share,density,area_sq_km,...,ind_79.0,ind_78.0,ind_35.0,ind_61.0,ind_84.0,ind_83.0,ind_55.0,ind_47.0,ind_33.0,ind_26.0
0,2016-08-05,83519024339,5,6,None,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,2016-08-19,85713599098,5,12,None,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,2016-08-17,89000000015,3,16,Tele2,2,0.0376,0.5432,73.045029,75485,...,False,False,False,False,False,False,False,False,False,False
3,2016-08-21,89000000101,7,16,Tele2,2,0.0376,0.5432,73.045029,75485,...,False,False,False,False,False,False,False,False,False,False
4,2016-08-14,89000003555,7,8,Tele2,2,0.0376,0.5432,73.045029,75485,...,False,False,False,False,False,False,False,False,False,False
5,2016-08-15,89000010423,1,16,Tele2,2,0.0376,0.5432,73.045029,75485,...,False,False,False,False,False,False,False,False,False,False
6,2016-08-02,89000075107,2,11,Tele2,2,0.0376,0.5432,73.045029,75485,...,False,False,False,False,False,False,False,False,False,False
7,2016-08-12,89000099858,5,15,Tele2,2,0.0376,0.5432,73.045029,75485,...,False,False,False,False,False,False,False,False,False,False
8,2016-08-08,89000099852,1,17,Tele2,2,0.0376,0.5432,73.045029,75485,...,False,False,False,False,False,False,False,False,False,False
9,2016-08-14,89000100151,7,17,Tele2,36,0.0091,0.8197,1.694383,787633,...,False,False,False,False,False,False,False,False,False,False


In [11]:
for c in [c for c in feat_cols if (not c in new_data.columns) and not 'XGBoost' in c]:
    new_data[c] = 0

NameError: name 'feat_cols' is not defined

In [19]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean')

feat_cols1 = [c for c in new_data.columns if not c in [u'ymd',u'phone_num',u'approve',u'full_app',u'stop_cond',
                u'target',u'no_need', u'not_started',u'util_30',u'status',u'mob_provider','federal_district','ind']
             and  not 'ind' in c]
#label_col = 'no_need'
imp.fit(new_data[new_data['ymd'] == '2016-08-01'][feat_cols1])
Xtt = imp.transform(new_data[feat_cols1])

In [20]:
XGB

{'XGBoost_approve': XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
        min_child_weight=1, missing=nan, n_estimators=100, nthread=-1,
        objective='binary:logistic', reg_alpha=0, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=True, subsample=1),
 'XGBoost_full_app': XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
        min_child_weight=1, missing=nan, n_estimators=100, nthread=-1,
        objective='binary:logistic', reg_alpha=0, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=True, subsample=1),
 'XGBoost_no_need': XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
        min_child_weight=1, missing=nan, n_estimators=100, nthread=-1,
        objective='binary:logistic', reg_alpha=0, reg_lamb

In [21]:
for k in ['XGBoost_no_need','XGBoost_not_started','XGBoost_target']:
    new_data[k] = [e[1] for e in XGB[k].predict_proba(Xtt)]
#new_data['no_need' + '_XGBoost']= [e[1] for e in clfXGB1.predict_proba(Xtt)]
#new_data['not_started' + '_XGBoost']= [e[1] for e in clfXGB2.predict_proba(Xtt)]
#new_data['target' + '_XGBoost']= [e[1] for e in clfXGB3.predict_proba(Xtt)]

In [23]:
feat_cols1 = [c for c in new_data.columns if not c in [u'ymd',u'phone_num',u'approve',u'full_app',u'stop_cond',
                u'target',u'no_need', u'not_started',u'util_30',u'status',u'mob_provider','federal_district','ind']
             and  not 'ind' in c]
#label_col = 'no_need'
imp.fit(new_data[new_data['ymd'] == '2016-08-01'][feat_cols1])
Xtt = imp.transform(new_data[feat_cols1])
new_data['full_app' + '_XGBoost']= [e[1] for e in XGB['XGBoost_full_app'].predict_proba(Xtt)]


In [ ]:
print(sklearn.metrics.roc_auc_score(
                  y_true = test_data['full_app'], 
                  y_score = [e[1] for e in clfXGB4.predict_proba(imp.transform(test_data[feat_cols1]))]
                )                                              
)

In [25]:
feat_cols1 = [c for c in new_data.columns if not c in [u'ymd',u'phone_num',u'approve',u'full_app',u'stop_cond',
                u'target',u'no_need', u'not_started',u'util_30',u'status',u'mob_provider','federal_district','ind']
             and  not 'ind' in c]
#label_col = 'no_need'
imp.fit(new_data[new_data['ymd'] == '2016-08-01'][feat_cols1])
Xtt = imp.transform(new_data[feat_cols1])
new_data['approve' + '_XGBoost']= [e[1] for e in XGB['XGBoost_approve'].predict_proba(Xtt)]

In [83]:
print(sklearn.metrics.roc_auc_score(
                  y_true = test_data['approve'], 
                  y_score = [e[1] for e in clfXGB5.predict_proba(imp.transform(test_data[feat_cols1]))]
                )                                              
)

0.534919386255


In [29]:
cols_to_export = [c for c in new_data.columns if not 'ind_' in c and not 'mob_provider_' in c and not 'federal_district_' in c]
cols_to_export

['ymd',
 'phone_num',
 'weekday',
 'raw_hour',
 'mob_provider',
 'ind',
 'pop_country_share',
 'pop_city_share',
 'density',
 'area_sq_km',
 'federal_district',
 'avg_salary_2015_rub',
 'utc_time_zone_val',
 'XGBoost_no_need',
 'XGBoost_not_started',
 'XGBoost_target',
 'full_app_XGBoost',
 'approve_XGBoost']

In [ ]:
cols_to_export = [c for c in train_data.columns if not 'ind_' in c and not 'mob_provider_' in c and not 'federal_district_' in c]
new_data[cols_to_export].to_csv('./data/new_data_ccalls1.csv',encoding = 'utf-8')

In [30]:
cols_to_export = [c for c in new_data.columns if not 'ind_' in c and not 'mob_provider_' in c and not 'federal_district_' in c]

hc.sql('drop table if exists new_data1')
hc.registerDataFrameAsTable(hc.createDataFrame(new_data[cols_to_export]), 'new_data1')
hc.sql('drop table if exists user_kposminin.ccalls_phone_data_only_test_2')
hc.sql('create table user_kposminin.ccalls_phone_data_only_test_2 as select * from new_data1')

DataFrame[]